id2: all those from English literal translation prompt that were comprehended.

uniques : do not have an equivalent in Darija as an idiom

id1, idioms: a part of idioms that are not unique

original: all the idioms

In [1]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

df = pd.read_csv('id2.csv')
def equivalent(id):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Language translation from Moroccan Darija dialect to English. " +
            "You will be provided with an idiom in Moroccan Darija dialect, "+
            "and your task is to translate it into an equivalent idiom or construction in English with no explanations."},
            {"role": "user", "content": "طلع تاكول الكرموس, نزل شكون قالها ليك."},
            {"role": "assistant", "content": "Caught between a rock and a hard place."},
            {"role": "user", "content":  f"{id}" }
            ], 
        temperature=0
        )
    return completion.choices[0].message.content

Index(['literal_eng', 'equivalent_eng', 'darija_ar', 'explanation',
       'Unnamed: 4'],
      dtype='object')


In [4]:
from sentence_transformers import SentenceTransformer, util
import plotly.graph_objects as go

data = pd.read_csv('id2.csv')

model_name = 'bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)


def evaluate_pairs(transl, eng):
    results = []
    for i in range(len(transl)):
        sentence1, sentence2 = transl[i], eng[i]
        embedding1 = model.encode(sentence1, convert_to_tensor=True)
        embedding2 = model.encode(sentence2, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
        results.append((i+2, sentence1, sentence2, similarity))

    return results

eng_equivalent = data['eng_equivalent']
eng_literal = data['eng_literal']
darija_equivalent =  data['darija_equivalent']
darija_literal =  data['darija_equivalent']

equivalent = data['equivalent']
literal = data['literal']

eng_similarities_eq = evaluate_pairs(eng_equivalent, equivalent)
eng_percentages_eq = [result[3] for result in eng_similarities_eq]

eng_similarities_lit = evaluate_pairs(eng_literal, literal)
eng_percentages_lit = [result[3] for result in eng_similarities_lit]

darija_similarities_eq = evaluate_pairs(darija_equivalent, equivalent)
darija_percentages_eq = [result[3] for result in darija_similarities_eq]

darija_similarities_lit = evaluate_pairs(darija_literal, literal)
darija_percentages_lit= [result[3] for result in darija_similarities_lit]

percentages = pd.DataFrame()
percentages['eng_equivalent'] = eng_percentages_eq
percentages['eng_literal'] = eng_percentages_lit
percentages['darija_equivalent'] = darija_percentages_eq
percentages['darija_literal'] = darija_percentages_lit

percentages.describe()


['طلع تاكول الكرموس, نزل شكون قالها ليك.', 'أش خاصك يا العريان, خاصني خاتم الذهب يا مولاي.', 'ماديرش يدك في غيران باش مايعضوكش الحنوشة.', 'ماديرش راسك في نخالة باش ماينقبوكش الدجاج.', 'اللي عضو الحنش كيخاف من الحبل.', 'جا يكحلها عماها.', 'اللي ما عندو فلوس كلامو مسوس.', 'دابزو يا الفيران على خميرة الجيران.', 'امدح صاحبك مع الناس ولومو الراس فالراس.', 'دير عين شافت وعين ماشافت.', 'دير عين ميكا.', 'ما تيحس بالمزود غير اللي مخبوط به.', 'خصك معاهم الخشيبات.', 'الفقيه دحمان، قرا البرة سبع ايام، وقاليهم الخط عيان.', 'مول التاج كيحتاج.', 'مية تخميمة وتخميمة ولا ضربة بالمقص.', 'اللي بغي لعسل يصبر لقريص النحل.', 'الگنازة كبيرة والميت فار.', 'اللي قال العصيدة باردة، ادير يدو فيها.', 'ما يحكك اضهري غير ظفري.', 'قالو شمتك، قالو جربتك.', 'دوز على الواد الهرهوري ولا دوز على الواد السكوتي.', 'اللي تربي على شي حاجة كبر عليها.', 'لي حفر شي حفرة كيطيح فيها.', 'الناس في الناس والقرع في مشيط الراس.', 'شحال من الفيل قرصاتوا عقرب.', 'الدنيا دراعة كل واحد كيلبسها ساعة.', 'الزين إلا زين الفعال.', 'زواج ليلة ت

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Box(y=percentages['eng_equivalent'], name='English equivalent translation prompt'))
fig.add_trace(go.Box(y=percentages['eng_literal'], name='English literal translation prompt'))
fig.add_trace(go.Box(y=percentages['darija_equivalent'], name='Darija equivalent translation prompt'))
fig.add_trace(go.Box(y=percentages['darija_literal'], name='Darija literal translation prompt'))

# Update layout
fig.update_layout(
    title='Box Plot of Similarity Scores for all the prompts.',
    yaxis_title='Similarity Score',
    xaxis_title='Prompt',
    boxmode='group'  # group the box plots together
)
# Show plot
fig.show()

In [1]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

df = pd.read_csv('id2.csv')

def equivalent(id):
    literal_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Language translation from Moroccan Darija dialect to English. " +
            "You will be provided with a sentence in Moroccan Darija dialect, "+
            "and your task is to translate it into an English sentence. "+
            "The translation must be literal with no explanations."},
            #{"role": "assistant", "content": "OK, I understand. I am ready for the translation."},
            {"role": "user", "content": "طلع تاكول الكرموس, نزل شكون قالها ليك."},
            {"role": "assistant", "content": "Come upstairs and have some figs, get downstairs who told you so!"},
            {"role": "user", "content":  f"{id}" }
        ], 
        temperature=0  # Deterministic responses
    )
    literal = literal_completion.choices[0].message.content

    # Step 1: Conclude a possible explanation for the given Darija idiom
    explanation_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will receive a literal English translation of a Moroccan Darija idiom, "+
            "and your task is to determine its possible meaning."},
            {"role": "user", "content": "Come upstairs and have some figs, get downstairs who told you so!"},
            {"role": "assistant", "content": "The idiom is about someone who's been told to do something but then he gets blamed after he does it."},

            {"role": "user", "content": f"{literal}" }
        ], 
        temperature=0  # Deterministic responses
    )
    explanation = explanation_completion.choices[0].message.content
    
    # Step 2: Give an equivalent idiom from the explanation
    equivalent_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will be provided with an explanation of an idiom, "+
            "and your task is to provide the equivalent English idiom or construction for it."},
            {"role": "user", "content": "The idiom is about someone who's been told to do something but then he gets blamed after he does it."},
            {"role": "assistant", "content": "Caught between a rock and a hard place"},
            {"role": "user", "content": f"{explanation}" }
        ], 
        temperature=0  # Deterministic responses
    )
    equivalent_idiom = equivalent_completion.choices[0].message.content
    
    # Step 3: Revise output of the translation before outputting it
    revision_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will receive a possible English idiom, " +
            "and your task is to revise it for accuracy and appropriateness. " +
            "If it fits as an idiom, keep it; if not, provide a suitable alternative."},
            {"role": "user", "content": "Caught between a rock and a hard place."},
            {"role": "assistant", "content": "Caught between a rock and a hard place."},
            {"role": "user", "content": f"{equivalent_idiom}" }
        ], 
        temperature=0  # Deterministic responses
    )
    revised_idiom = revision_completion.choices[0].message.content
    
    return literal, explanation,equivalent_idiom,revised_idiom

# Example usage


In [5]:
darija_idiom = "أش خاصك يا العريان, خاصني خاتم الذهب يا مولاي."
print(f"Literal translation: {equivalent(darija_idiom)[0]}")
print(f"Its meaning: {equivalent(darija_idiom)[1]}")
print(f"equivalent in English: {equivalent(darija_idiom)[2]}")
print(f"Recheck: {equivalent(darija_idiom)[3]}")


Literal translation: What do you want, you naked one, give me the gold ring, sir.
Its meaning: This idiom means that someone is asking for something unreasonable or impossible.
equivalent in English: To ask for the moon
Recheck: To ask for the moon


It's not systematic for multiple turns